In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import unicodedata

In [2]:
data=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8",index_col=0)
data.loc[data["メーカー名"].isna()].shape
data.loc[data["メーカー名"].isna(),"医薬品名"].value_counts()
len(set(data.loc[data["メーカー名"].isna(),"医薬品名"]))


C:\Users\hakos\AppData\Local\Temp\ipykernel_39208\3130333295.py:1: DtypeWarning: Columns (118) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8",index_col=0)


52

In [3]:

#n2n
name2name_ii = pd.read_pickle('./pickle/name2name_ii.pkl')
name2name_nk = pd.read_pickle('./pickle/name2name_nk.pkl')
name2name_cl_nk=pd.read_pickle('./pickle/name2name_cl_nk.pkl')
name2name=pd.read_pickle('./pickle/name2name.pkl')

#c2f
code2firm=pd.read_pickle('./pickle/code2firm.pkl')
# n2f
name2firm_ii = pd.read_pickle('./pickle/name2firm_ii.pkl')
name2firm_nk = pd.read_pickle('./pickle/name2firm_nk.pkl')
# name2firm_ma=pd.read_pickle('./pickle/name2firm_ma.pkl')
name2firm=pd.read_pickle('./pickle/name2firm.pkl')

# cl nk
cleaned2firm=pd.read_pickle('./pickle/cleaned2firm_nk.pkl')
name2cleaned=pd.read_pickle('./pickle/name2cleaned_nk.pkl')

# name2firm_cl_nk = {name: cleaned2firm[cleaned] for name, cleaned in name2cleaned.items()}
# update 
# name2name_nk.update(name2name)
name2name_nk.update(name2name_ii)
name2name_nk.update(name2name_cl_nk)

# name2firm_nk.update(name2firm)
name2firm_nk.update(name2firm_ii)
name2firm_nk.update(cleaned2firm)

# make them list
name2name_ii=list(name2name_ii.items())
name2name_nk=list(name2name_nk.items())
name2name_cl_nk=list(name2name_cl_nk.items())
len(name2name_ii),len(name2name_nk)


(25, 52)

In [4]:
# name2firm_nk

In [5]:
cnt=0
def delete_ok(name2name,delete_no=0):
    delete_list=[]
    # keyとvalueについて、「」内が同じかどうかを確認する
    for i,(key,value) in enumerate(name2name):
        cleaned_key = unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', key).group()) if re.search(r'.*\u3000', key) else key)
        #空白を削除
        cleaned_key = cleaned_key.replace(' ','')
        # print(cleaned_key,value,cleaned_key==value)
        if cleaned_key==value or key==value:
            # cnt+=1
            delete_list.append(i)
            continue
        try:
            pattern = r'「(.*?)」'
            key_v=re.findall(pattern, key)[0]
            key_v = ''.join([unicodedata.normalize('NFKC', char) for char in key_v])
            value_v=re.findall(pattern, value)[0]
            value_v = ''.join([unicodedata.normalize('NFKC', char) for char in value_v])
            # print(key_v,value_v,key_v==value_v)
            
            if key_v==value_v:
                delete_list.append(i)
        except:
            continue
            # remove key and value
    # delete
    if delete_no>0:
        name2name=[name2name[i] for i in range(len(name2name)) if i in delete_list]
    else:
        name2name = [name2name[i] for i in range(len(name2name)) if i not in delete_list]
    return name2name
name2name_ii=delete_ok(name2name_ii,delete_no=1)
name2name_nk=delete_ok(name2name_nk,delete_no=1)
name2name_cl_nk=delete_ok(name2name_cl_nk,delete_no=1)
len(name2name_ii),len(name2name_nk),len(name2name_cl_nk)

(25, 51, 14)

In [6]:
# for x in dict(name2name_ii).keys():
#     print(x,data.loc[data["医薬品名"]==x,"医薬品コード"].unique().astype(int))

In [7]:
# for x in dict(name2name_nk).keys():
#     print(x,data.loc[data["医薬品名"]==x,"医薬品コード"].unique().astype(int))

In [8]:
name2name_nk=dict(name2name_nk)
name2firm = {key: name2firm_nk[key] for key in name2firm_nk if key in name2name_nk}
len(name2firm)

51

In [9]:
data["医薬品コード"]=data["医薬品コード"].astype(int).astype(str)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_nk)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_ii)
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm_ma)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2cleaned).map(cleaned2firm)
# data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品コード"].map(code2firm)
data_lack=data[data["メーカー名"].isna()]
print(len(set(data_lack["医薬品名"])))
data.to_csv("../merged/len3_ndb_blp_DN_firm_val.csv",encoding="utf-8")

0


In [10]:
set(data_lack["医薬品名"])

set()

医薬品名に対してレセプト電算コードが一意でないもの

In [11]:

# data=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
# data["医薬品コード"]=data["医薬品コード"].astype(int)
# name_set=set(data["医薬品名"])
# for name in name_set:
#     code_set=set(data.loc[data["医薬品名"]==name,"医薬品コード"])
#     # yjコードが変わってるのは結構ある
#     yj_code_set=set(data.loc[data["医薬品名"]==name,"薬価基準収載医薬品コード"])
#     maker_set=set(data.loc[data["医薬品名"]==name,"メーカー名"])
#     if len(code_set)!=1:
#         print(name,code_set,yj_code_set,maker_set)

統一収載でした


- ビホナゾール１％クリーム {622331600, 622781000}

- クロトリマゾール１％クリーム {622330200, 622780800} クロトリマゾールクリーム1％「イワキ」岩城製薬

- ジクロフェナクナトリウム５０ｍｇ坐剤 {622330800, 622849300}